In [124]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from sklearn.preprocessing import LabelEncoder
%matplotlib inline

In [125]:
os.chdir("/Users/carolineroper/Documents/School/Machine Learning/Reset_Project/noda")

In [126]:
import project_env as pe

In [136]:
#bring in BOFI_NBR, SCREENING_DISP_CODE
data_simple = pd.read_csv('data_processing/output/data_simple.csv', encoding = "ISO-8859-1", low_memory=False)
data_simple = data_simple[['SCREENING_DISP_CODE','UNIQUE_ID','BOFI_NBR']]

In [137]:
data_simple.head()

,SCREENING_DISP_CODE,UNIQUE_ID,BOFI_NBR
0,NaN,0,255544
1,NaN,1,257683
2,NaN,2,255696
3,NaN,3,251021
4,NaN,4,246712


In [138]:
bin_features = pd.read_csv('data_processing/output/df_bin_features.csv', encoding = "ISO-8859-1", low_memory=False)
num_features = pd.read_csv('data_processing/output/df_num_features.csv', encoding = "ISO-8859-1", low_memory=False)
date_features = pd.read_csv('data_processing/output/df_date_features.csv', encoding = "ISO-8859-1", low_memory=False) \
                            .drop('JUVENILE_FLAG',axis=1)
cat_features = pd.read_csv('data_processing/output/df_cat_features.csv', encoding = "ISO-8859-1", low_memory=False)
rearrest = pd.read_csv('data_processing/output/df_rearrest_times.csv', encoding = "ISO-8859-1", low_memory=False)

print('bin_features: %s' %(str(rearrest.shape)))
print('num_features: %s' %(str(num_features.shape)))
print('date_features: %s' %(str(date_features.shape)))
print('cat_features: %s' %(str(cat_features.shape)))
print('rearrest: %s' %(str(rearrest.shape)))

bin_features: (269543, 2)
num_features: (280294, 5)
date_features: (272088, 21)
cat_features: (275692, 9)
rearrest: (269543, 2)


In [141]:
merged = pd.merge(rearrest, \
                 bin_features,\
                 on='UNIQUE_ID', \
                 how='left')
merged = pd.merge(merged, \
                 num_features,\
                 on='UNIQUE_ID', \
                 how='left')
merged = pd.merge(merged, \
                 date_features,\
                 on='UNIQUE_ID', \
                 how='left')
merged = pd.merge(merged, \
                 cat_features,\
                 on='UNIQUE_ID', \
                 how='left')
merged = pd.merge(merged, \
                 data_simple,\
                 on='UNIQUE_ID', \
                 how='left')

In [131]:
merged.to_csv('merged_pj.csv', index=False)

In [260]:
merged.columns

Index(['UNIQUE_ID', 'NEXT_ARREST_TIME', 'ARREST_DATE', 'ARREST_DATE_y',
       'BOFI_NBR', 'SCREENING_DISP_CODE', 'BAR_ADMIT_DAYS', 'CRIMINAL_FLAG',
       'FINAL_DETENTION_FLAG', 'HABITUAL_OFFENDER_FLAG',
       'INITIAL_DETENTION_FLAG', 'JUVENILE_FLAG', 'SADA_SEX', 'SEX',
       'TOT_NUM_DEF', 'MULTIPLE_DEF_FLAG', 'SCREENING_DAYS',
       'SCREENING_DISP_DATE_y', 'SCREENING_DISP_DATE_m', 'AGE',
       'ARREST_TO_SCREEN', 'CHARGE_CLASS', 'CHARGE_TYPE', 'CHARGE_DESC',
       'PARTY', 'RACE', 'SADA_RACE', 'AGE_JUV_INVALID', 'AGE_NA'],
      dtype='object')

In [142]:
merged.CHARGE_CLASS.value_counts(dropna=False)

FELONY, OPTIONAL                  116732
MISDEMEANOR                        97363
FELONY, HARD LABOR                 43597
JUVENILE STATUS                     6138
NaN                                 4271
CAPITAL                             1345
MULTIPLE CLASS CHARGES IN CASE        70
ASSET FORFEITURE                      27
Name: CHARGE_CLASS, dtype: int64

In [144]:
len(merged.CHARGE_DESC.value_counts(dropna=False))

157

In [149]:
merged.CHARGE_DESC.isnull().sum()/merged.shape[0] #4% are missing

0.041388572509766533

In [146]:
merged.CHARGE_DESC.value_counts(dropna=False) #11156 nulls

POSS SCHEDULE 2 OTHER                             26146
POSS MARIJUANA 1ST OFFENSE                        18406
POSS STOLEN PROP OVER 500                         14712
CURFEW FOR PERSONS UNDER 17; SCHOOL ATTENDANCE    11973
DIST SCHED 2 NARCOTIC                             11518
NaN                                               11156
AGGRAVATED BATTERY                                10350
SIMPLE BURGLARY                                    8763
SIMPLE BATTERY                                     8194
ARMED ROBBERY                                      7223
THEFT OF GOODS UNDER 100                           7155
THEFT OVER 500                                     6399
THEFT BETWEEN 100 AND 500                          6104
THEFT UNDER 100                                    5698
CONCEALED WEAPON FIRST                             5434
FORGERY                                            5410
AGGRAVATED ASSAULT                                 5027
CRIMINAL TRESPASS                               

In [150]:
cols = ['UNIQUE_ID', 'NEXT_ARREST_TIME', 'ARREST_DATE', 'ARREST_DATE_y','BOFI_NBR','SCREENING_DISP_CODE',\
        'BAR_ADMIT_DAYS','CRIMINAL_FLAG', \
        'FINAL_DETENTION_FLAG', 'HABITUAL_OFFENDER_FLAG', \
        'INITIAL_DETENTION_FLAG', 'JUVENILE_FLAG', 'SADA_SEX', \
        'SEX', 'TOT_NUM_DEF', 'MULTIPLE_DEF_FLAG', 'SCREENING_DAYS', \
        'SCREENING_DISP_DATE_y', \
        'SCREENING_DISP_DATE_m', 'AGE', 'ARREST_TO_SCREEN', \
        'CHARGE_CLASS', 'CHARGE_TYPE', 'CHARGE_DESC', 'PARTY', 'RACE', \
        'SADA_RACE','AGE_JUV_INVALID','AGE_NA']

merged = merged[cols]

In [151]:
merged.to_csv('merged_pj.csv',index=False)

In [152]:
#identify arrests where at least one charge was accepted
accepted = merged[merged['SCREENING_DISP_CODE']==230][['BOFI_NBR','ARREST_DATE']]

In [153]:
#identify arrests where at least one charge was accepted
accepted = merged[merged['SCREENING_DISP_CODE']==230][['BOFI_NBR','ARREST_DATE']]

#classify rows where at least one charge was accepted during that arrest
classified = pd.merge(merged, \
                 accepted, \
                 on=['BOFI_NBR','ARREST_DATE'], \
                 how='outer',\
                 indicator = True)

classified['ACCEPTED'] = np.where(classified['_merge']=='both', 1, 0)

classified = classified.drop('_merge', axis=1)

#remove duplicate arrests on same day
#confirmed that it's possible that an arrest has at least one charge was accepted and is still flagged as "delete"
classified = classified[classified['NEXT_ARREST_TIME']!='Delete']

#convert NEXT_ARREST_TIME to numeric
classified['NEXT_ARREST_TIME'] = classified['NEXT_ARREST_TIME'].apply(pd.to_numeric)

In [154]:
cat_var = ['SADA_SEX', 'SEX', 'PARTY', 'RACE', 'SADA_RACE', \
           'SCREENING_DISP_DATE_y', 'SCREENING_DISP_DATE_m', 'CHARGE_TYPE', \
           'CHARGE_CLASS', 'CHARGE_DESC', 'ARREST_DATE_y']

cat_var_df = pd.DataFrame()

for column in cat_var:
    category = np.repeat(classified.loc[:,column].name, len(classified.loc[:,column].value_counts(dropna=False)))
    value = classified.loc[:,column].value_counts(dropna=False).index
    count = classified.loc[:,column].value_counts(dropna=False)
    cat_var_df = cat_var_df.append(pd.DataFrame(np.transpose(np.vstack([category, value, count]))))
cat_var_df.columns = ['Category', 'Value', 'Count']
cat_var_df['Count'] = pd.to_numeric(cat_var_df['Count'])

cat_var_df.fillna(value = "Missing", inplace=True)

cat_var_df['Label'] = cat_var_df['Category'] + '_' + cat_var_df['Value']

In [155]:
#two rounds of splits so predicted judge training set includes all training records from risk prediction model and so on.

test_ne_1, train_ne_1, val_ne_1 = pe.split_data(classified[classified['ACCEPTED']==0], test_split=.2, \
                                 train_split=.64, by_var='ARREST_DATE_y', random_state=1)

test_ne_2, train_ne_2, val_ne_2 = pe.split_data(classified[classified['ACCEPTED']==1], test_split=.2, \
                                 train_split=.64, by_var='ARREST_DATE_y', random_state=1)

test_ne = pd.concat([test_ne_1, test_ne_2],axis=0)
train_ne = pd.concat([train_ne_1, train_ne_2],axis=0)
val_ne = pd.concat([val_ne_1, val_ne_2],axis=0)

Test Data 25068
(100272, 30)
Val Data 20055
Train Data 80217
Test Data 21078
(84308, 30)
Val Data 16862
Train Data 67446


In [156]:
train_ne.to_csv('data_train_pj.csv', index=False)
test_ne.to_csv('data_test_pj.csv', index=False)
val_ne.to_csv('data_val_pj.csv', index=False)

In [252]:
cat_var_enc = pe.one_hot_encode(classified[cat_var])
cat_var_enc = pd.DataFrame(cat_var_enc.toarray(), index=classified.index) #can pass the column names here - 

In [158]:
after_enc_df = pd.DataFrame()

for column in list(cat_var_enc.columns):
    after_enc_df = after_enc_df.append(pd.DataFrame([column,sum(cat_var_enc.loc[:,column])]).transpose())
after_enc_df.columns = ['current_name', 'Count']

In [168]:
classified[cat_var].columns

Index(['SADA_SEX', 'SEX', 'PARTY', 'RACE', 'SADA_RACE',
       'SCREENING_DISP_DATE_y', 'SCREENING_DISP_DATE_m', 'CHARGE_TYPE',
       'CHARGE_CLASS', 'CHARGE_DESC', 'ARREST_DATE_y'],
      dtype='object')

In [253]:
#checked several columns by the number with each value and confirmed this is the pattern the encoder uses
col_names = []
for col in cat_var:
    types = list(np.sort(classified[col].unique().astype(str)))
    types = [col + '_' + x for x in types]
    types = [x.replace(' ', '_') for x in types]
    col_names.extend(types)

In [254]:
cat_var_enc.columns = col_names

In [255]:
classified_enc = classified.drop(cat_var[:-1], axis=1)
classified_enc = pd.merge(classified_enc, \
                       cat_var_enc,\
                       left_index=True, \
                       right_index=True, \
                       how='left')

In [256]:
classified_enc.head()

,UNIQUE_ID,NEXT_ARREST_TIME,ARREST_DATE,ARREST_DATE_y,BOFI_NBR,SCREENING_DISP_CODE,BAR_ADMIT_DAYS,CRIMINAL_FLAG,FINAL_DETENTION_FLAG,HABITUAL_OFFENDER_FLAG,...,ARREST_DATE_y_1990.0,ARREST_DATE_y_1991.0,ARREST_DATE_y_1992.0,ARREST_DATE_y_1993.0,ARREST_DATE_y_1994.0,ARREST_DATE_y_1995.0,ARREST_DATE_y_1996.0,ARREST_DATE_y_1997.0,ARREST_DATE_y_1998.0,ARREST_DATE_y_1999.0
1,5183,0,1988-01-03,1988.0,290600,NaN,1952,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,5073,3209,1988-01-03,1988.0,290588,NaN,1952,1,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,104025,1794,1988-01-03,1988.0,220453,NaN,1952,1,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,18252,516,1988-01-03,1988.0,J78755,230.0,1952,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,6411,0,1988-01-03,1988.0,X010116,230.0,2284,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [257]:
#two rounds of splits so predicted judge training set includes all training records from risk prediction model and so on.

test_1, train_1, val_1 = pe.split_data(classified_enc[classified_enc['ACCEPTED']==0], test_split=.2, \
                                 train_split=.64, by_var='ARREST_DATE_y', random_state=1)

test_2, train_2, val_2 = pe.split_data(classified_enc[classified_enc['ACCEPTED']==1], test_split=.2, \
                                 train_split=.64, by_var='ARREST_DATE_y', random_state=1)

test = pd.concat([test_1, test_2],axis=0)
train = pd.concat([train_1, train_2],axis=0)
val = pd.concat([val_1, val_2],axis=0)

#went from 93 columns to 250 columns b/c of the charge description

Test Data 25068
(100272, 250)
Val Data 20055
Train Data 80217
Test Data 21078
(84308, 250)
Val Data 16862
Train Data 67446


In [258]:
train.to_csv('train_pj.csv',index=False)
val.to_csv('val_pj.csv',index=False)
test.to_csv('test_pj.csv', index=False)